In [1]:
!pip install streamlit transformers requests wordcloud matplotlib
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.4 MB/s eta 0:00:00


In [10]:
%%writefile  Personalized_News_Aggregator.py
import streamlit as st
import requests
from transformers import pipeline
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Initialize summarizer
summarizer = pipeline("summarization")

# News API configuration
API_KEY = 'fb45776498bf4b2eb6dde8222abfca27'
BASE_URL = 'https://newsapi.org/v2/everything'

def fetch_news(query, num_articles=5):
    """Fetch news articles using NewsAPI."""
    params = {
        'q': query,
        'apiKey': API_KEY,
        'language': 'en',
        'pageSize': num_articles
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        return response.json()['articles']
    else:
        error_message = response.json().get('message', 'Unknown error')
        st.error(f"Failed to fetch news. Error: {response.status_code} - {error_message}")
        return []

def summarize_article(content):
    """Summarize the article content using HuggingFace pipeline."""
    if len(content.split()) > 50:
        summary = summarizer(content, max_length=100, min_length=30, do_sample=False)
        return summary[0]['summary_text']
    return content

def generate_wordcloud(text):
    """Generate a word cloud from the provided text."""
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    return wordcloud

# Streamlit app
st.title("📰 Personalized News Aggregator")

# Input: User query
query = st.text_input("Enter a topic or keyword to search for news:", "AI")
num_articles = st.slider("Number of articles to fetch:", 1, 10, 5)

if st.button("Fetch News"):
    st.write(f"### Top {num_articles} Articles on '{query}'")

    # Fetch and display articles
    articles = fetch_news(query, num_articles)
    combined_text = ""

    for i, article in enumerate(articles):
        st.subheader(f"{i + 1}. {article['title']}")
        st.write(f"**Source:** {article['source']['name']}")
        st.write(f"**Published At:** {article['publishedAt']}")

        if 'content' in article and article['content']:
            summary = summarize_article(article['content'])
            st.write(f"**Summary:** {summary}")
            combined_text += article['content'] + " "
        else:
            st.write("No content available to summarize.")

        st.write(f"[Read Full Article]({article['url']})")

    # Generate word cloud
    if combined_text:
        st.write("### Word Cloud of Topics")
        wordcloud = generate_wordcloud(combined_text)
        fig, ax = plt.subplots()
        ax.imshow(wordcloud, interpolation='bilinear')
        ax.axis('off')
        st.pyplot(fig)

st.write("\n---")
st.write("Built with 🚀 by Asem")


Overwriting Personalized_News_Aggregator.py


In [3]:
!ngrok authtoken 2nOimZbLDMI3ZYBd4Lpez2D5qP2_428RBQAr8uhh49FZjPcSH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
from pyngrok import ngrok
!streamlit run Personalized_News_Aggregator.py&>/dev/nul&

In [5]:
!pgrep -f streamlit

6452


In [6]:
public_url=ngrok.connect(8501)

In [7]:
public_url

<NgrokTunnel: "https://b962-34-45-217-182.ngrok-free.app" -> "http://localhost:8501">